In [1]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class ClaimSourceDataset(Dataset):
    def __init__(self, df, collection_df, tokenizer, max_len=256):
        self.samples = []
        self.tokenizer = tokenizer
        self.max_len = max_len

        paper_lookup = collection_df.set_index('cord_uid')

        for _, row in df.iterrows():
            tweet = row['tweet_text']
            paper_id = row['cord_uid']

            if paper_id not in paper_lookup.index:
                continue

            paper_row = paper_lookup.loc[paper_id]
            paper = f"{paper_row['title']} {paper_row['abstract']}"

            paper_tokenized = tokenizer(
                paper,
                truncation=True,
                padding='max_length',
                max_length=max_len,
                return_tensors="pt",
                return_overflowing_tokens=True
            )

            tweet_tokenized = tokenizer(
                tweet,
                truncation=True,
                padding='max_length',
                max_length=max_len,
                return_tensors="pt",
                return_overflowing_tokens=True
            )

            # Handle multiple overflow chunks (if any) — flat storage
            for i in range(len(paper_tokenized['input_ids'])):
                for j in range(len(tweet_tokenized['input_ids'])):
                    self.samples.append({
                        'tweet_input_ids': tweet_tokenized['input_ids'][j].squeeze(0),
                        'tweet_attention_mask': tweet_tokenized['attention_mask'][j].squeeze(0),
                        'paper_input_ids': paper_tokenized['input_ids'][i].squeeze(0),
                        'paper_attention_mask': paper_tokenized['attention_mask'][i].squeeze(0)
                    })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


In [2]:
import torch
import torch.nn as nn
from transformers import AutoModel

class DualEncoder(nn.Module):
    def __init__(self, model_name="scibert_scivocab_uncased"):
        super().__init__()
        self.encoder = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')


    def forward(self, tweet_ids, tweet_mask, paper_ids, paper_mask):
        tweet_vec = self.encoder(tweet_ids, attention_mask=tweet_mask).last_hidden_state[:, 0]
        paper_vec = self.encoder(paper_ids, attention_mask=paper_mask).last_hidden_state[:, 0]
        return tweet_vec, paper_vec


In [3]:
import torch.nn.functional as F

def contrastive_loss(tweet_vecs, paper_vecs, temperature=0.05):
    tweet_vecs = F.normalize(tweet_vecs, dim=1)
    paper_vecs = F.normalize(paper_vecs, dim=1)

    logits = torch.matmul(tweet_vecs, paper_vecs.T) / temperature
    labels = torch.arange(len(tweet_vecs)).to(tweet_vecs.device)
    return F.cross_entropy(logits, labels)


In [4]:
import numpy as np
import pandas as pd

In [5]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = 'subtask4b_collection_data.pkl' #MODIFY PATH
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)


In [6]:
from transformers import AutoTokenizer

def compute_max_context_length(df_collection, df_query, max_rows=None):
    """
    Computes the maximum context length of [CLS] title + abstract [SEP] tweet [SEP]
    using the SciBERT tokenizer.
    
    Args:
        df_collection (pd.DataFrame): DataFrame containing 'cord_uid', 'title', and 'abstract'.
        df_query (pd.DataFrame): DataFrame containing 'cord_uid' and 'tweet_text'.
        max_rows (int or None): Optional limit on number of rows for speed. If None, use all.
        
    Prints:
        Maximum, average, and 95th percentile context lengths (in tokens).
    """
    tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
    lens = []
    
    # Create a dict for fast lookup of paper metadata
    paper_map = df_collection.set_index('cord_uid')[['title', 'abstract']].to_dict(orient='index')
    
    sample_rows = df_query if max_rows is None else df_query.sample(n=max_rows, random_state=42)

    for _, row in sample_rows.iterrows():
        tweet = str(row['tweet_text'])
        cord_uid = row['cord_uid']
        
        if cord_uid in paper_map:
            title = str(paper_map[cord_uid]['title'])
            abstract = str(paper_map[cord_uid]['abstract'])
            input_text = f"{title} {abstract}"
            inputs = tokenizer.encode_plus(input_text, tweet, truncation=False, add_special_tokens=True)
            lens.append(len(inputs['input_ids']))

    if not lens:
        print("No matching tweet-paper pairs found.")
        return

    print(f"Max length: {max(lens)} tokens")
    print(f"Average length: {sum(lens)/len(lens):.2f} tokens")
    print(f"95th percentile: {int(np.percentile(lens, 95))} tokens")

In [15]:
compute_max_context_length(df_collection, df_query_train)

Max length: 1795 tokens
Average length: 423.29 tokens
95th percentile: 770 tokens


In [14]:
df_collection

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055874,w094r17f,Medline; PMC,COVID-19 increased the risk of ICU-acquired bl...,10.1007/s00134-021-06346-w,PMC7839935,33506379,no-cc,PURPOSE: The primary objective of this study w...,2021-01-27,"Buetti, Niccolò; Ruckly, Stéphane; de Montmoll...",Intensive Care Med,NaN,NaN,NaN,w094r17f,2021-01-27,1611705600
1056107,wtqwghon,MedRxiv; WHO,Relative disease burdens of COVID-19 and seaso...,10.1101/2020.04.22.20073551,NaN,NaN,medrxiv,Comparisons between the mortality burdens of C...,2020-04-27,"Faust, Jeremy; del Rio, Carlos",NaN,NaN,NaN,NaN,wtqwghon,2020-04-27,1587945600
1056228,r5rctvul,Medline; PMC,COVID‐19 and the Gender Gap in Work Hours,10.1111/gwao.12506,PMC7361447,32837019,no-cc,School and daycare closures due to the COVID‐1...,2020-07-02,"Collins, Caitlyn; Landivar, Liana Christin; Ru...",Gend Work Organ,NaN,NaN,NaN,r5rctvul,2020-07-02,1593648000
1056344,yjqt74q0,Medline; PMC,"Prior infection by seasonal coronaviruses, as ...",10.2807/1560-7917.es.2021.26.13.2001782,PMC8017906,33797390,cc-by,BACKGROUND: Children have a low rate of COVID-...,2021-04-01,"Sermet-Gaudelus, Isabelle; Temmam, Sarah; Huon...",Euro Surveill,NaN,NaN,NaN,yjqt74q0,2021-04-01,1617235200


In [16]:
len(df_query_test['tweet_text'][:])

1446

In [7]:
PATH_QUERY_TRAIN_DATA = 'subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = 'subtask4b_query_tweets_dev.tsv' #MODIFY PATH
PATH_QUERY_DEV_TEST = 'subtask4b_query_tweets_test.tsv' #MODIFY PATH

In [8]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')
df_query_test = pd.read_csv(PATH_QUERY_DEV_TEST, sep = '\t')

In [10]:
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', return_overflowing_tokens=True)
model = DualEncoder().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

train_dataset = ClaimSourceDataset(df_query_train, df_collection, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16 , shuffle=False)

In [11]:
for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        tweet_ids = batch['tweet_input_ids'].to(device)
        tweet_mask = batch['tweet_attention_mask'].to(device)
        paper_ids = batch['paper_input_ids'].to(device)
        paper_mask = batch['paper_attention_mask'].to(device)

        tweet_vecs, paper_vecs = model(tweet_ids, tweet_mask, paper_ids, paper_mask)

        loss = contrastive_loss(tweet_vecs, paper_vecs)
   
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


100%|██████████| 1587/1587 [36:48<00:00,  1.39s/it]


Epoch 1 Loss: 1.1701


100%|██████████| 1587/1587 [36:11<00:00,  1.37s/it]


Epoch 2 Loss: 0.9617


100%|██████████| 1587/1587 [35:55<00:00,  1.36s/it]


Epoch 3 Loss: 0.8946


100%|██████████| 1587/1587 [36:01<00:00,  1.36s/it]


Epoch 4 Loss: 0.8783


100%|██████████| 1587/1587 [36:42<00:00,  1.39s/it]

Epoch 5 Loss: 0.8625


In [14]:
import faiss
import numpy as np

def encode_papers(model, df_collection, tokenizer, batch_size=8, max_len=256):
    model.eval()
    paper_texts = df_collection.apply(lambda row: f"{row['title']} {row['abstract']}", axis=1).tolist()
    paper_ids = df_collection['cord_uid'].tolist()

    all_embeddings = []
    all_paper_ids = []

    with torch.no_grad():
        for i in range(0, len(paper_texts), batch_size):
            batch_texts = paper_texts[i:i+batch_size]
            batch_ids = paper_ids[i:i+batch_size]

            encodings = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                return_overflowing_tokens=True,
                return_tensors="pt",
                max_length=max_len
            )

            input_ids = encodings['input_ids'].to(device)
            attention_mask = encodings['attention_mask'].to(device)
            overflow_map = encodings['overflow_to_sample_mapping']

            vecs = model.encoder(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0].cpu().numpy()

            for j, vec in enumerate(vecs):
                original_idx = overflow_map[j].item()
                all_embeddings.append(vec)
                all_paper_ids.append(batch_ids[original_idx])

    return all_paper_ids, np.vstack(all_embeddings)



In [15]:
paper_ids, paper_embeddings = encode_papers(model, df_collection, tokenizer)

faiss_index = faiss.IndexFlatIP(paper_embeddings.shape[1])
faiss_index.add(paper_embeddings)

paper_id_map = {i: pid for i, pid in enumerate(paper_ids)}


In [16]:
def retrieve(model, df_query_dev, tokenizer, faiss_index, paper_id_map, topk=10):
    model.eval()
    predictions = []

    with torch.no_grad():
        for text in df_query_dev['tweet_text']:
            enc = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            tweet_vec = model.encoder(enc['input_ids'], attention_mask=enc['attention_mask']).last_hidden_state[:, 0]
            tweet_vec = F.normalize(tweet_vec, dim=1).cpu().numpy()

            D, I = faiss_index.search(tweet_vec, topk)
            
            preds = [paper_id_map[idx] for idx in I[0]]
            predictions.append(preds)

    df_query_dev['dense_topk'] = predictions


In [18]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

# Evaluate
retrieve(model, df_query_dev, tokenizer, faiss_index, paper_id_map)
results_test = get_performance_mrr(df_query_dev, 'cord_uid', 'dense_topk')
print("MRR Results:", results_test)


MRR Results: {1: 0.4492857142857143, 5: 0.5170714285714286, 10: 0.5267610544217688}


In [ ]:

df_query_test['preds'] = df_query_test['dense_topk'].apply(lambda x: x[:5])
df_query_test[['post_id', 'preds']].to_csv('predictions_new_scibert_final_context.tsv', index=None, sep='\t')


In [13]:
# Save model weights
model_save_path = "dual_encoder_model.pt"
torch.save(model.state_dict(), model_save_path)


In [ ]:
from transformers import AutoTokenizer

# Reinitialize model and load weights
model_loaded = DualEncoder()
model_loaded.load_state_dict(torch.load("dual_encoder_model.pt"))
model_loaded.to(device)